In [ ]:
import pathlib
import matplotlib.pyplot as plt
import torch
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from utils.graph_utils_topo_change import prepare_dataset

In [ ]:
# Use some required pathes
DATA_PATH = pathlib.Path().resolve() / "input_data_local" / "lips_idf_2023"
BENCH_CONFIG_PATH = pathlib.Path().resolve() / "configs" / "benchmarks" / "lips_idf_2023.ini"

In [ ]:
benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               benchmark_name="Benchmark_competition_DC",
                               load_data_set=False, # we set False, as data are not yet generated
                               config_path=BENCH_CONFIG_PATH,
                               log_path=None)

## Generate some data using DC solver

In [ ]:
benchmark.generate(nb_sample_train=int(2e3),
                   nb_sample_val=int(2e3),
                   nb_sample_test=int(2e3),
                   nb_sample_test_ood_topo=int(2e3),
                   store_as_sparse=True, # NOTE: Specific to envs with 118 or more nodes
                   do_store_physics=True,
                   is_dc=True
                   )

In [ ]:
print(benchmark.train_dataset.size)
print(benchmark.val_dataset.size)
print(benchmark._test_dataset.size)
print(benchmark._test_ood_topo_dataset.size)

In [ ]:
device = torch.device("cpu") # or "cuda:0" if you have any GPU
train_loader, val_loader, test_loader, test_ood_loader = prepare_dataset(benchmark=benchmark, batch_size=2, device=device)

In [ ]:
batch = next(iter(train_loader))

In [ ]:
batch

In [ ]:
from utils.graph_utils_topo_change import GPGmodel_without_NN

device = torch.device("cpu") # select "cuda:0" if you have a GPU
gpg_model_wo_nn = GPGmodel_without_NN(ref_node=68, num_gnn_layers=2000, device=device)
gpg_model_wo_nn.to(device)

In [ ]:
import numpy as np

predictions_list = []
observations_list = []
error_per_batch = []
for batch in test_loader:
    out, errors = gpg_model_wo_nn(batch)
    predictions_list.append(out)
    observations_list.append(batch.y)
    error_per_batch.append([float(error.detach().cpu().numpy()) for error in errors])
observations = torch.vstack(observations_list)
predictions = torch.vstack(predictions_list)
errors = np.vstack(error_per_batch)
errors = errors.mean(axis=0)

In [ ]:
plt.figure()
plt.plot(errors)
plt.xlabel("# GNN layers (iterations)")
plt.ylabel("Local Conservation Error")

In [ ]:
import math
predictions = predictions * (180/math.pi) * 100.

In [ ]:
np.concatenate((predictions, observations), axis=1)[:100]

In [ ]:
from lips.metrics.ml_metrics.external_metrics import mape_quantile

MAPE_10 = mape_quantile(y_true=observations.detach().cpu(), y_pred=predictions.detach().cpu(), quantile=0.9)
print("MAPE 10: ", MAPE_10)

In [ ]:
import warnings
from utils.graph_utils import get_obs
from utils.graph_utils_topo_change import get_all_active_powers

warnings.filterwarnings("ignore")
env, obs = get_obs(benchmark)
p_ors_pred, p_exs_pred = get_all_active_powers(benchmark._test_dataset.data,
                                               obs,
                                               theta_bus=predictions.view(-1,obs.n_sub*2))

In [ ]:
# put the predictions in a dictionary in order to that the LIPS framework could read them for evaluation
my_predictions = {}
my_predictions["p_or"] = p_ors_pred
my_predictions["p_ex"] = p_exs_pred

In [ ]:
MAPE10_Power = mape_quantile(y_true=benchmark._test_dataset.data["p_or"], 
                             y_pred=my_predictions["p_or"],
                             quantile=0.9)
print("MAPE10 on Active Powers: ", MAPE10_Power)

In [ ]:
from lips.metrics.power_grid.local_conservation import local_conservation

LC_tolerance = 1e-2

verification = local_conservation(predictions=my_predictions,
                                  observations=benchmark._test_dataset.data,
                                  tolerance=LC_tolerance,
                                  env=env,
                                  result_level=2)
print(f"Violation percentage: {verification['violation_percentage']:.2f} %")